![Podmanlogo](Pictures/podman-logo.png)

# What is Podman?

As described by the Podman website itself it is:

    Podman is a daemonless, open source, Linux native tool designed to make it easy to find, run, build, share and deploy applications using Open Containers Initiative (OCI) Containers and Container Images. Podman provides a command line interface (CLI) familiar to anyone who has used the Docker Container Engine. Most users can simply alias Docker to Podman (alias docker=podman) without any problems. Similar to other common Container Engines (Docker, CRI-O, containerd), Podman relies on an OCI compliant Container Runtime (runc, crun, runv, etc) to interface with the operating system and create the running containers. This makes the running containers created by Podman nearly indistinguishable from those created by any other common container engine.

With simple words it's a container engine that is so similar to Docker that you can just use an alias and run docker-like commands. This brings to mind an obvious question, if it's so similar to Docker why would you use Podman instead? The answer is security.
The previous paragraph starts describing Podman as a "daemonless" container engine and that's the main difference with Docker. Later in this workshop we'll explain security features in Podman, for the moment just keep in mind that as Podman doesn't have a daemon it doesn't need to use the root user for anything, therefore the execution of containers is secured by design.

This is explained differently in Podman website:

    Containers under the control of Podman can either be run by root or by a non-privileged user. Podman manages the entire container ecosystem which includes pods, containers, container images, and container volumes using the libpod library. Podman specializes in all of the commands and functions that help you to maintain and modify OCI container images, such as pulling and tagging. It allows you to create, run, and maintain those containers and container images in a production environment.

Moreover, podman has gone further and brings some new functionalities that will help enterprises looking at container adoption for their workloads. We will explain some of these during this workshop.

# Why Podman?

Many think of Podman to be a replacement for Docker (if they have heard of Podman at all). But, this is not the case, as Podman is another option that provides better security and developer features. Podman is a cloud-native, daemonless tool that helps developers manage their Linux containers. Podman is all about security, but also minimizing the friction between your local development environment and production.

Podman uses a microservices approach, creating a network with many other cloud-native products, such as Buildah and Skopeo, to build and push containers. This makes Podman a lighter and faster application than Docker, allowing for customization and changes.

# Podman hello world: pulling and executing your first container image

Just like any other container engine, Podman has the ability to manage and execute container images. A container image is a static image that contains an applications with all it's dependencies. Because the dependencies are included within the container image we can expect the same application behaviour if it's executed in different hosts, solving one of the most widespread problems for developers: "it worked in my environment".

Container images are stored in container registries, by default Podman will look for container images in quay.io, registry.redhat.io and docker.io. You can configure additional registries by modifying the file /etc/containers/registries.conf, we'll not cover this advanced configuration during this workshop.

The way Podman works with container images is it locally downloads (or pulls) images from registry. Hence, first thing we want to do is checking if we have any image downloaded locally:

In [ ]:
%login student{{ STDID }}@ {{ IP-WKSHP-Podman101 }}

In [ ]:
!podman images

You should have no images locally. Download your first image, the podman hello-world image, by running the following command:

In [ ]:
!podman pull hello-world

Now check if your image was pulled correctly by running again the podman images command:

In [ ]:
!podman images

You should find the hello-world image is locally downloaded in your system. This only means that we have a static image that contains our application and its dependancies locally sync'ed. Next thing we would like to do is to execute that static image:

In [ ]:
!podman run hello-world

# Executing containerized applications

During last exercise we downloaded a container image and executed it. With the following command Podman will show the containers that are being executed right now:

In [ ]:
!podman ps

As you can see, the output is empty. This is because the container that we executed runs a process that ends after printing in the screen the "Hello Podman World" message and once the main process of a container is finished the container itself will stop being executed.
But no worries, Podman got us covered, if we want to see all the containers that are being executed and those whose execution finished some time ago we just need to add the "-a" to the previous command.

In [ ]:
!podman ps -a

There you can see the container we executed before with a status of "Exited".

Lets see what happens if we execute a container whose process don't finish unless specified:

In [ ]:
!podman run nginx

You need to stop the process manually as the container process has taken all of your terminal. If you want to run a container in the background, then you should add the option --detach (or simply -d).

In [ ]:
!podman run --detach nginx

Now, if you run the "podman ps" command you will see your nginx container with a status of "Up".

In [ ]:
!podman ps

You see the container has an ID and also a name. In this case the name was randomly generated as we did not specify any name, pay attention to next time we execute a container image as we will be giving it a name.

You can stop all running containers by running the following:

In [ ]:
!podman stop --all

Now that all containers have beeen stopped run again "podman ps" to check if all of them were correctly stopped.

In [ ]:
!podman ps

As you can see the container is not present because "podman ps" only shows the containers that are being executed right now. If we use the "-a" option it will show all containers that are in execution right now but also those that were stopped, it'll show the stopped ones with an status of "Exited".

In [ ]:
!podman ps -a

This means that containers are not deleted once the process is finished and you could simply restart any of them. We can do this by using the container ID, but for the sake of simplicity we will be creating a new container with a defined name:

In [ ]:
!podman run -d --name my-podman-container nginx
!podman ps

See how we changed the long "--detach" option for a short "-d" but it works the same way. We also defined the name. So now we can just stop it by it's name.

In [ ]:
!podman stop my-podman-container
!podman ps -a

Now that it is stopped, we can restart it.

In [ ]:
!podman start my-podman-container
!podman ps

While the container is running you can also inspect it to get to know all the information about it.

In [ ]:
!podman inspect my-podman-container

If we stop it again we see that we can also remove it by running the command "podman rm"

In [ ]:
!podman stop my-podman-container
!podman rm my-podman-container

Now it has been fully removed so we wont be able to start that same container again, we will need to create a new container from the same image. You can check it is fully removed by looking at the empty output of "podman ps -a"

In [ ]:
!podman ps -a

A very common question is how to modify the configuration of a running container. There are several ways you can do this, one of the most common is to run the container interactively. You will need to add two options for this purpose, first one is the "--tty" that will allocate a pseudo-TTY for the container; the second is "--interactive" which will keep STDIN open even if not attached. We can combine bot options just by using the "-it" option.
Then you will need to specify the command you want to run in your container. If you specify "/bin/bash" as the command you want to run you will open an interactive terminal with your container.

In our case, this environment based on jupyter notebooks doesn't work well with interactive commands. This is why instead of using "/bin/bash" command we will use the command "sleep 300" that will our container as a process running for the next 300 seconds.

In [ ]:
!podman run -it -d --name my-podman-container fedora sleep 300

If the container is already running you can just use the "podman exec" command and then specify the command that you want to run. Again, if you use "/bin/bash" you'll have access to the shell inside the container but you can just run any other command as shown in the example below.

In [ ]:
!podman exec -it my-podman-container cat /etc/fedora-release

As you can see the output showed the version of Fedora running within the container.

# Cleanup

In [ ]:
!podman stop --all
!podman rm --all
!podman rmi hello-world nginx fedora


In [ ]:
%logout